# Cryptocurrency Agent with Azure AI Agent Service

This notebook demonstrates how to interact with a Cryptocurrency Agent via the Azure AI SDK. It is designed to guide you step-by-step through the process, including:

- Setting up environment variables and dependencies
- Enabling telemetry and tracing
- Creating agent threads and sending messages
- Polling run status and extracting the final answer

Follow along as each cell explains a part of the process.

## Step 1: Setup and Dependencies

In this step, we install the prerequisite libraries and load environment variables.

In [1]:
%pip install azure-ai-projects azure-identity
%pip install opentelemetry-api azure-monitor-opentelemetry 
%pip install python-dotenv rich

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [1]:
# Import dependencies and load environment variables
import os
import time
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from dotenv import load_dotenv
from rich import print  # for rich formatted output

load_dotenv()

# Initialize the project client using connection string from environment
project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(),
    conn_str=os.environ["AZURE_AI_AGENT_PROJECT_CONNECTION_STRING"],
)

print("[green]Dependencies loaded and client initialized.[/green]")

Dependencies loaded and client initialized.

## Step 2: Enable Tracing and Telemetry

Here, we enable Azure Monitor tracing and additional instrumentation. This ensures all operations of the agent are monitored.

In [2]:
# Import tracing libraries
from opentelemetry import trace
from azure.monitor.opentelemetry import configure_azure_monitor

# Enable Azure Monitor tracing
application_insights_connection_string = project_client.telemetry.get_connection_string()
if not application_insights_connection_string:
    print("[bold red]Application Insights was not enabled for this project.[/bold red]")
    print("[bold red]Enable it via the 'Tracing' tab in your AI Foundry project page.[/bold red]")
    exit()

configure_azure_monitor(connection_string=application_insights_connection_string)

# Enable additional instrumentations
project_client.telemetry.enable()

# Setup a tracer for the current scenario
import os
scenario = os.path.basename(__file__) if '__file__' in globals() else 'notebook_session'
tracer = trace.get_tracer(__name__)

print("[green]Tracing and telemetry are enabled.[/green]")

Tracing and telemetry are enabled.

## Step 3a: Agent Initialization and Creating a Thread

Next, we initialize the Cryptocurrency Agent and create a new thread to communicate with it.

In [11]:
with tracer.start_as_current_span(scenario):
    # Initialize the agent using the assistant ID from environment variables
    agent = project_client.agents.get_agent(
        assistant_id=os.environ["AZURE_AI_CRYPTO_AGENT_ID"]
    )
    print(f"[green]Fetching Cryptocurrency Agent with ID: {agent.id}[/green]")

    # Create a new thread
    thread = project_client.agents.create_thread()
    print(f"[green]Created thread, thread ID: {thread.id}[/green]")

    # Create a message with the query content
    message = project_client.agents.create_message(
        thread_id=thread.id,
        role="user",
        content=(
            "Find out the 5 most traded cryptocurrencies today."
            "If that currency was traded on various exchanges,"
            "calculate the sum of all trading volumes. "
            "Also list the exchanges traded on, in a separate column."
        ),
    )
    print(f"[green]Created message, message ID: {message.id}[/green]")

Fetching Cryptocurrency Agent with ID: asst_4bPhvgIUvIkmBwOpgLpfDIDe

Created thread, thread ID: thread_FHO7vORgAGTpZxj8dkFgDMoW

Created message, message ID: msg_LDvYWU9cpc9c2hlXpPQhRTxD

## Step 4a: Running the Agent and Extracting the Final Answer

Here, we trigger the agent run, poll for its status until completion, and finally extract and print the final assistant answer.

In [12]:
    # Create a run for the agent
    run = project_client.agents.create_run(thread_id=thread.id, assistant_id=agent.id)

    # Poll the run as long as the run status is queued or in progress
    while run.status in ["queued", "in_progress", "requires_action"]:
        time.sleep(10)
        run = project_client.agents.get_run(thread_id=thread.id, run_id=run.id)
        print(f"[yellow]Run status: {run.status}[/yellow]")

    # Get all messages and print only the final assistant answer
    messages = project_client.agents.list_messages(thread_id=thread.id)
    final_answer = next(
        (msg['content'][0]['text']['value'] for msg in messages['data']
         if msg['role'] == 'assistant' and msg.get('assistant_id')),
        "No answer received"
    )
    print(f"[green]Final answer: {final_answer}[/green]")

Run status: in_progress

Run status: in_progress

Run status: in_progress

Run status: in_progress

Run status: in_progress

Run status: completed

Final answer: Here are the five most traded cryptocurrencies today along with their total trading volumes and 
exchanges they were traded on:

| Cryptocurrency | Total Trading Volume (USD) | Exchanges Traded On                                                
|
|----------------|----------------------------|--------------------------------------------------------------------
-----------------------|
| Bitcoin        | $70,697.76                 | Binance, BVOX, Bybit, OKX, CoinW, Crypto.com Exchange, Bitget, 
AscendEX (BitMax), DigiFinex, HTX, P2B, Hotcoin, BitMart, Coinbase Exchange, Dex-Trade |
| Tether         | $1,519,623,175.72           | Binance, Bitget, KuCoin, Bybit, DigiFinex, Pionex                 
|
| Ethereum       | $1,235,044.72              | Binance, Pionex, Bitget, Bybit, KuCoin                             
|
| Ripple         | $80,815,788.99             | Binance, Bybit, MEXC, OKX, DigiFinex                               
|
| Litecoin       | $757,435.84                | OKX, MEXC, Bybit, Bitget, Coinbase Exchange                        
|

### Summary of Execution Steps:
1. Retrieved trading information including volumes and exchanges for the five most traded cryptocurrencies today.
2. Collected the volume data for each currency from multiple exchanges.
3. Calculated the total trading volume for each cryptocurrency.
4. Compiled the results into a concise table format for better readability.

## Step 3b: Agent Initialization and Creating a Thread with new task

Let us ask a new question to the agent. We will create a new thread with a different task and run it.

In [11]:
with tracer.start_as_current_span(scenario):
    # Initialize the agent using the assistant ID from environment variables
    agent = project_client.agents.get_agent(
        assistant_id=os.environ["AZURE_AI_CRYPTO_AGENT_ID"]
    )
    print(f"[green]Fetching Cryptocurrency Agent with ID: {agent.id}[/green]")

    # Create a new thread
    thread = project_client.agents.create_thread()
    print(f"[green]Created thread, thread ID: {thread.id}[/green]")

    # Create a message with the query content
    message = project_client.agents.create_message(
        thread_id=thread.id,
        role="user",
        content=(
            "Find the OHLCV data for BTC for the last 12 hours and " +
            "plot a candlestick chart with it to provide a quick snapshot. " +
            "Make sure you plot full OHLCV data to yield a complete candlestick chart. "
        ),
    )
    print(f"[green]Created message, message ID: {message.id}[/green]")

Fetching Cryptocurrency Agent with ID: asst_4bPhvgIUvIkmBwOpgLpfDIDe

Created thread, thread ID: thread_kXO7mTRwvQ3enhNSYzesOXA5

Created message, message ID: msg_cEJ6dbqF6qgICtmZ2AcOJCfe

## Step 4b: Running the Agent and Extracting the Final Answer

Let us trigger the new agent run, poll for its status until completion, and finally extract and print the final assistant answer.

In [12]:
    # Create a run for the agent
    run = project_client.agents.create_run(thread_id=thread.id, assistant_id=agent.id)

    # Poll the run as long as the run status is queued or in progress
    while run.status in ["queued", "in_progress", "requires_action"]:
        time.sleep(10)
        run = project_client.agents.get_run(thread_id=thread.id, run_id=run.id)
        print(f"[yellow]Run status: {run.status}[/yellow]")

    # Get all messages and print only the final assistant answer
    messages = project_client.agents.list_messages(thread_id=thread.id)
    
    # Print Assistant Messages
    for msg in messages['data']:
            for item in msg['content']:
                if item.get('type') == 'text':
                    text = item.get('text', {}).get('value')
                    if text:
                        print(f"[yellow]{text}[/yellow]")

    # If candlestick image is present, store it
    for msg in messages['data']:
        for item in msg['content']:
            if item.get('type') == 'image_file':
                image_info = item.get('image_file', {})
                file_id = image_info.get('file_id')
                if file_id:
                    # Fetch the file content as binary data
                    image_file = project_client.agents.get_file_content(file_id=file_id)
                    # Display the image via iPython
                    img_data = b"".join(list(image_file))
                    with open("candlestick.png", "wb") as f:
                        f.write(img_data)


Run status: in_progress

Run status: in_progress

Run status: in_progress

Run status: in_progress

Run status: completed

The candlestick chart for BTC/USD over the last 12 hours has been displayed successfully. 

### Summary of Execution Steps:
1. **Data Retrieval**: Gathered OHLCV data for Bitcoin (BTC) covering the last 12 hours.
2. **Data Preparation**: Created a Pandas DataFrame from the OHLCV data and converted timestamps to a datetime 
format suitable for plotting.
3. **Custom Plotting Function**: Defined a function to manually plot a candlestick chart using Matplotlib, since 
`mplfinance` was not available in the environment.
4. **Plotting Execution**: Used the defined function to display the candlestick chart, showing bullish and bearish 
candlesticks along with high, low, open, and close prices.

If you have any further questions or need additional analyses, feel free to ask!

Find the OHLCV data for BTC for the last 12 hours and plot a candlestick chart with it to provide a quick snapshot.
Make sure you plot full OHLCV data to yield a complete candlestick chart. 

## Summary

This notebook showcased how to integrate with and monitor a Cryptocurrency Agent using the Azure AI SDK. We configured telemetry, created a communication thread, sent a query, and finally extracted the agent's final response. Execute the cells in sequence for a comprehensive demonstration.